In [3]:
import pandas as pd
import os
import ast
import time
from tqdm import tqdm
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Set max column width
pd.set_option("max_colwidth", 500)

In [4]:
# Input directory (dir. with csvs containing parsed articles)
DF_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\temp_isna_notna_base\


In [5]:
DF_output = input().strip()

 D:\Data_40_years_cancer_studies\BERT_NER_parsedXMLs_temp_isna_notna_base\


In [6]:
list_csvs = []

for file in os.listdir(DF_input):
    if file[-4:] == ".csv":
        list_csvs.append(file)

n_csvs = len(list_csvs)

In [8]:
# Input directory for the BERT model fine tuned for affiliation NER
path_BERT = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\bert-fineutned-ner-4511-optuna-optimized\


In [9]:
# Load BERT model
model = AutoModelForTokenClassification.from_pretrained(path_BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy='max')

Device set to use cpu


In [10]:
# Extract the affiliation of the last author (or, if that is not available, of any author) of each article
# Then, do NER on the affiliation, add result to dataframe and save it
no_affiliation = []
parsed_csvs = []

for csv in list_csvs:
    if csv not in parsed_csvs: 
        start = time.time()
        NER_input = {} # PMIDs used as keys, affiliation for this PMID as values
        NER_output = {} # PMIDs used as keys, NER from BERT as values

        # Part 1: Extract affiliation of last (or, if not available, any other) author of each article
        print(f"Extracting affiliation of articles in csv: {csv} ({str(list_csvs.index(csv)+1)}/{str(n_csvs)})")
        df= pd.read_csv(DF_input + csv)
        for i in range(df.index[-1] + 1): # Iterate through all rows, including the last one
            NER_input[df.at[i, "PMID"]] = []
            try:
                authors_data = ast.literal_eval(df.at[i, "Authors"])
                for author in authors_data:
                    if "Affiliation" in author and author["Affiliation"]:
                        for affiliation in author["Affiliation"]:
                            NER_input[df.at[i, "PMID"]].append(affiliation)

                if len(NER_input[df.at[i, "PMID"]]) == 0:
                    no_affiliation.append(df.at[i, "PMID"])

            except Error:
                NER_input[df.at[i, "PMID"]] = None
                no_affiliation.append(df.at[i, "PMID"])

        for key in NER_input.keys():
            NER_input[key] = list(set(NER_input[key]))
    
            if NER_input[key] == []:
                NER_input[key] == None
    
            else:
                combined_input = ""
                for affiliation in NER_input[key]:
                    combined_input += ". " + affiliation
        
                NER_input[key] = combined_input[2:]

        # Part 2: Do NER on the affiliation to extract structured info
        for key in tqdm(NER_input.keys()):
            if len(NER_input[key]) > 0:
                text = NER_input[key]
                entities = nlp(text)
                NER_output[key] = entities

            else:
                NER_output[key] = []

    df_NER_BERT = pd.DataFrame(pd.Series(NER_output, name="values"))
    df_NER_BERT["PMID_NER"] = df_NER_BERT.index
    df_NER_BERT = df_NER_BERT.rename(columns={"values": "NER_BERT"}).reset_index().drop(columns=["index"])

    df_save = pd.merge(df, df_NER_BERT, left_on= "PMID", right_on="PMID_NER", how="left")
    df_save = df_save.drop(columns=["PMID_NER"])

    df_save.to_csv(DF_output + "BERT_NER_" + csv, index = False)

Extracting affiliation of articles in csv: temp_isna_notna.csv (1/1)


100%|████████████████████████████████████████████████████████████████████████| 141701/141701 [2:03:21<00:00, 19.14it/s]
